# **Data analisys for validation MODIS vs VESPA-22 IWV measurements**

## **Selected data reception from Thule-mix-LAADS.ipynb**

In [ ]:
%store -r max_IWV
%store -r max_dist_from_THAAO
%store -r deltatime
%store -r new_mod_vespa
%store -r new_myd_vespa
%store -r new_mxd_vespa

### **Controllo dati ricevuti**

In [ ]:
num_mod_data_p_Dt = len(new_mod_vespa.IWV_MODIS) 
num_myd_data_p_Dt = len(new_myd_vespa.IWV_MODIS)
num_mxd_data_p_Dt = len(new_mxd_vespa.IWV_MODIS)
num_modmyd_data_p_Dt = num_mod_data_p_Dt + num_myd_data_p_Dt

In [ ]:
print(f'-- Selection parameters --')
print(f'max IWV = {max_IWV} [mm]')
print(f'max distance from THAOO = {max_dist_from_THAAO} [km]')
print(f'Deltatime chosen for data points selection = {deltatime}\n')
print(f'-- Data points selected --')
print(f'Selected number of MOD data points =          {num_mod_data_p_Dt}')
print(f'Selected number of MYD data points =          {num_myd_data_p_Dt}\n')
print(f'Total selected number of MXD data points     =  {num_mxd_data_p_Dt}')
print(f'Total selected number of MOD+MYD data points =  {num_modmyd_data_p_Dt}\n')